# VITS-JaPros-WebUI 訓練colabノートブック
[VITS-JaPros-WebUI](https://github.com/litagin02/vits-japros-webui)の訓練パートのみを行うノートブックです。

リポジトリのローカル使用では、`data`フォルダ（学習元データを入れるところ）と`outputs`フォルダ（途中結果や訓練結果等を入れるところ）がありますが、このノートブックではGoogle driveの`vits-japros-workspace`フォルダ内にこれらのフォルダを作って、そこで作業を行います。

## 補足

- 音声データのうち5ファイルは学習データとして使われず、検証データとして使われます。
- どれだけの音声データがあれば質が良くなるか等は分かりません、実験してください。

# 0. 環境構築
今のcolab環境だとtensorflowの整合性エラーが出ることがあるみたいだけどたぶん問題ないはず。

In [ ]:
!git clone https://github.com/litagin02/vits-japros-webui
%cd vits-japros-webui
!pip install -r requirements.txt
!curl -L "https://huggingface.co/litagin/vits-japros-pretrained/resolve/main/pretrained.pth" -o "pretrained/pretrained.pth"

 ## 1. データセットの準備
- Google drive内の`vits-japros-workspace`フォルダで以下は作業します。

In [ ]:
# Google driveのマウント
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# フォルダ変数の設定
import os

root_dir = "/content/drive/MyDrive/vits-japros-workspace/"
data_dir = os.path.join(root_dir, "data")
wavs_dir = os.path.join(data_dir, "wavs")
split_wavs_dir = os.path.join(data_dir, "split_wavs")
output_dir = os.path.join(root_dir, "outputs")

os.makedirs(wavs_dir, exist_ok=True)

- 以下、フォルダは「Google drive内の`vits-japros-workspace`」を起点に考えてください。上のセルで、その中に`data/wavs/`フォルダがGoogle driveで作られたはずです。
- `data/wavs/`フォルダ内に、学習に用いる音声wavファイルを入れてください。ファイル名は空白を含まない半角英数字にしてください。また**44.1kHzでモノラル**なことを前提とします。過程で自動的に変換されますが、その際に音質が落ちる可能性があります。

- 既存コーパスなどでセリフ文章がすでにある場合は、`data/transcript_utf8.txt`ファイルに、音声ファイルのファイル名（拡張子以外）と、その音声のテキスト、半角コロン`:`区切りで書いてください。書き起こしが無い場合は、次のステップでの自動書き起こしを利用します。

例：

```txt
wav_file1:これは最初の音声です。
next_wav:これはもしかして、次のファイルの音声だったりする？
third:そうかもしれないにゃー。
...
```

## 2. 自動書き起こし

`data/transcript_utf8.txt`を作成済みの場合は、このステップは不要です。

[faster-whisper](https://github.com/guillaumekln/faster-whisper) を利用して、音声データから自動的に書き起こしを行います。

2つのオプションがあります。

1. 通常オプション: `data/wavs/`フォルダ内のwavファイル1つ1つをそのまま書き下します。
2. 分割オプション: `data/wavs/`フォルダ内のwavファイルそれぞれを、文が区切れている箇所（`。`等）で区切って、音声ファイルも分割し`data/split_wavs`に保存し、書き下しも分割します。学習の精度が上がるかもしれませんが、**変な箇所で区切られて発話が途切れる可能性等もあります**。結果を確認することをおすすめします。

結果は`data/transcript_utf8.txt`に上書き保存されます。

In [ ]:
# 通常オプションの場合
!python transcribe.py --data-dir {data_dir}

In [ ]:
# 分割オプションの場合
!python transcribe_split.py --data-dir {data_dir}

TIPS:

余裕があれば、手動で音声ファイルを聞きながら`data/transcript_utf8.txt`を修正しましょう：
- 不適切な音声（言葉にできない変な声・感情が激しすぎる声・途中で途切れている声等）があれば、その行を削除する（wavファイルはそのままで構いません）
- 誤字脱字修正
- 語尾が上がる疑問口調は、その箇所にちゃんと`？`を入れる
- ポーズ位置に`、`や`。`を入れ、逆にポーズがないところには`、`等を削除する
- 読み方が複数あって曖昧なものは、ひらがな等にする（「何で→なんで」「行った→おこなった」）

が、こだわりすぎなくても大丈夫かもしれません、よく分かりません。

## 3. 事前準備
- `model_name`は、好きな名前（半角英数字）を指定してください。学習結果は`outputs/{model_name}/checkpoints/`フォルダ内に保存されます。

In [ ]:
model_name = "test_model"  # ここにモデル名を入力

In [ ]:
# 通常オプションを使った場合
!python preprocess.py --model-name {model_name} --data-dir {data_dir} --wavs-dir {wavs_dir} --output-dir {output_dir}

In [ ]:
# 分割オプションを使った場合
!python preprocess.py --model-name {model_name} --data-dir {data_dir} --wavs-dir {split_wavs_dir} --output-dir {output_dir}

## 4. 学習

- `model_name`は上で指定したものと同じものを指定してください。
- `max_epoch`は、最大学習エポック数を指定します。デフォルトは200です。
- `batch_bins`は、学習時のバッチサイズのようなものを指定します。デフォルトは1000000です。多いほどグラボのVRAM使用量は上がりますが、学習データ量によりどの程度まで上げられるかは変わるみたいです。グラボのVRAMに合わせてはみ出ないようにしてください。

In [ ]:
max_epoch = 200
batch_bins = 1000000
# model_name = "test_model"  # 3をスキップして途中から学習を再開するとき

下のセルを実行すると学習が開始されます（たぶん通常は終わらない）。

- 結果は1エポックごとに`vits-japros-workspace/outputs/{model_name}/checkpoints/`の`{数字}epoch.pth`に保存されています。
- 途中結果や学習に必要なデータはすべてGoogle driveの`vits-japros-workspace/outputs/`に保存されているので、途中から学習を再開するには、「0. 環境構築」と「1. データセットの準備」を行った後、そのまま「4. 学習」を実行できます。

In [ ]:
# 学習の実行
!python train.py --model-name {model_name} --max-epoch {max_epoch} --batch-bins {batch_bins} --output-dir {output_dir}